## 1. Download a Sample Filing

We will use a real N-CSR filing to test the parsers.

In [2]:
import os
import sys
import importlib

# Ensure project root is on sys.path so 'sec_extractor' can be imported
repo_root = os.path.abspath(os.path.join(os.getcwd(), '..'))
if repo_root not in sys.path:
    sys.path.insert(0, repo_root)

module_name = 'sec_extractor.core.http_client'
try:
    # Import or reload the module so tests pick up local changes
    if module_name in sys.modules:
        importlib.reload(sys.modules[module_name])
    else:
        importlib.import_module(module_name)

    # Import the exported instance
    from sec_extractor.core.http_client import http_client
    print('http_client loaded:', type(http_client))
except Exception as e:
    # Provide a readable error for quick debugging
    print('Failed to load http_client:', repr(e))
    raise


http_client loaded: <class 'sec_extractor.core.http_client.SECHTTPClient'>


In [2]:
# 3. Download a sample filing and save full + 50k sample
accession_url = "https://www.sec.gov/Archives/edgar/data/1084380/0001193125-24-194739.txt"
print('Downloading:', accession_url)
try:
    filing_text = http_client.get_text(accession_url, retries=3)
except Exception as e:
    print('Download failed:', repr(e))
    raise

print('Downloaded length:', len(filing_text))

# Save a 50k-character sample to avoid OOM in parsers
sample_len = 50000
filing_sample = filing_text[:sample_len]
print('Sample length:', len(filing_sample))

out_dir = os.path.join(os.getcwd(), 'notebook_outputs')
os.makedirs(out_dir, exist_ok=True)
full_path = os.path.join(out_dir, 'filing_full.txt')
sample_path = os.path.join(out_dir, 'filing_sample.txt')
with open(full_path, 'w', encoding='utf-8') as f:
    f.write(filing_text)
with open(sample_path, 'w', encoding='utf-8') as f:
    f.write(filing_sample)

print('Saved full to:', full_path)
print('Saved sample to:', sample_path)


Downloading: https://www.sec.gov/Archives/edgar/data/1084380/0001193125-24-194739.txt
Downloaded length: 98330376
Sample length: 50000
Downloaded length: 98330376
Sample length: 50000
Saved full to: c:\Users\DataBridge\Documents\workspace\Edgar\notebooks\notebook_outputs\filing_full.txt
Saved sample to: c:\Users\DataBridge\Documents\workspace\Edgar\notebooks\notebook_outputs\filing_sample.txt
Saved full to: c:\Users\DataBridge\Documents\workspace\Edgar\notebooks\notebook_outputs\filing_full.txt
Saved sample to: c:\Users\DataBridge\Documents\workspace\Edgar\notebooks\notebook_outputs\filing_sample.txt


In [6]:
import os
import time
import sys

# --- Cargar la muestra ---
sample_path = os.path.join(os.getcwd(), 'notebook_outputs', 'filing_sample.txt')
if not os.path.exists(sample_path):
    raise FileNotFoundError(f"El archivo de muestra no se encontró en: {sample_path}")

with open(sample_path, 'r', encoding='utf-8') as f:
    sample_text = f.read()

print(f"Muestra cargada: {len(sample_text)} caracteres.")
print("-" * 30)

# --- Prueba 1: secxbrl ---
print("Ejecutando prueba para 'secxbrl'...")
try:
    import secxbrl
    t0 = time.time()
    xbrl_out = secxbrl.parse_inline_xbrl(sample_text)
    t1 = time.time()
    
    print(f"  - Parseo completado en {t1 - t0:.3f} segundos.")
    print(f"  - Tipo de resultado: {type(xbrl_out)}")
    print(f"  - Elementos encontrados: {len(xbrl_out) if hasattr(xbrl_out, '__len__') else 'N/A'}")

except ImportError:
    print("  - ERROR: 'secxbrl' no está instalado.")
except Exception as e:
    print(f"  - ERROR durante el parseo de secxbrl: {repr(e)}")

print("-" * 30)

# --- Prueba 2: secsgml (usando la función correcta) ---
print("Ejecutando prueba para 'secsgml'...")
try:
    # Forzar la ubicación del paquete
    venv_site_packages = r"c:\Users\DataBridge\Documents\workspace\Edgar\venv\Lib\site-packages"
    if venv_site_packages not in sys.path:
        sys.path.insert(0, venv_site_packages)
    
    # Limpiar el caché de importación si existe
    if 'secsgml' in sys.modules:
        del sys.modules['secsgml']
    
    # Importar la función correcta
    from secsgml import parse_sgml_content_into_memory
    print("  - ¡Importación exitosa!")
    
    t0 = time.time()
    # Usar la función correcta con el texto como bytes
    sgml_out = parse_sgml_content_into_memory(sample_text.encode('utf-8'))
    t1 = time.time()

    print(f"  - Parseo completado en {t1 - t0:.3f} segundos.")
    print(f"  - Tipo de resultado: {type(sgml_out)}")

    # Inspeccionar el resultado
    if hasattr(sgml_out, 'documents'):
        print(f"  - Documentos encontrados: {len(sgml_out.documents)}")
        if len(sgml_out.documents) > 0:
            first_doc = sgml_out.documents[0]
            print(f"    - Tipo del primer documento: {getattr(first_doc, 'type', 'N/A')}")
            print(f"    - Atributos del primer documento: {[attr for attr in dir(first_doc) if not attr.startswith('_')][:10]}")
    elif hasattr(sgml_out, '__len__'):
        print(f"  - Elementos encontrados: {len(sgml_out)}")
    elif hasattr(sgml_out, '__dict__'):
        print(f"  - Atributos del resultado: {list(sgml_out.__dict__.keys())}")
    else:
        print(f"  - Vista previa del resultado: {repr(sgml_out)[:300]}...")

except Exception as e:
    print(f"  - ERROR durante el parseo de secsgml: {repr(e)}")

print("-" * 30)
print("Pruebas de parsers finalizadas.")

Muestra cargada: 50000 caracteres.
------------------------------
Ejecutando prueba para 'secxbrl'...
  - Parseo completado en 0.001 segundos.
  - Tipo de resultado: <class 'list'>
  - Elementos encontrados: 0
------------------------------
Ejecutando prueba para 'secsgml'...
  - ¡Importación exitosa!
  - Parseo completado en 0.003 segundos.
  - Tipo de resultado: <class 'tuple'>
  - Elementos encontrados: 2
------------------------------
Pruebas de parsers finalizadas.


In [7]:
# Celda 6: Análisis detallado de los resultados de secsgml
import os
import time
import sys

print("=== ANÁLISIS DETALLADO DE RESULTADOS ===")
print()

# --- Recargar muestra y ejecutar secsgml ---
sample_path = os.path.join(os.getcwd(), 'notebook_outputs', 'filing_sample.txt')
with open(sample_path, 'r', encoding='utf-8') as f:
    sample_text = f.read()

# Forzar importación de secsgml
venv_site_packages = r"c:\Users\DataBridge\Documents\workspace\Edgar\venv\Lib\site-packages"
if venv_site_packages not in sys.path:
    sys.path.insert(0, venv_site_packages)

if 'secsgml' in sys.modules:
    del sys.modules['secsgml']

from secsgml import parse_sgml_content_into_memory

# Ejecutar parser
sgml_out = parse_sgml_content_into_memory(sample_text.encode('utf-8'))

print(f"Resultado de secsgml:")
print(f"  - Tipo: {type(sgml_out)}")
print(f"  - Longitud: {len(sgml_out)}")
print()

# Inspeccionar cada elemento de la tupla
for i, element in enumerate(sgml_out):
    print(f"Elemento {i + 1}:")
    print(f"  - Tipo: {type(element)}")
    
    if hasattr(element, '__len__') and not isinstance(element, str):
        print(f"  - Longitud: {len(element)}")
    
    # Si es un objeto con atributos
    if hasattr(element, '__dict__'):
        attrs = [attr for attr in dir(element) if not attr.startswith('_')]
        print(f"  - Atributos: {attrs[:10]}")  # Primeros 10 atributos
        
        # Intentar mostrar algunos atributos clave
        for attr in ['type', 'name', 'content', 'text', 'documents', 'sections']:
            if hasattr(element, attr):
                val = getattr(element, attr)
                if isinstance(val, (str, int, float)):
                    print(f"    - {attr}: {val}")
                elif hasattr(val, '__len__'):
                    print(f"    - {attr}: {type(val)} con {len(val)} elementos")
                else:
                    print(f"    - {attr}: {type(val)}")
    
    # Si es una lista o diccionario, mostrar una muestra
    elif isinstance(element, (list, tuple)):
        print(f"  - Primeros 3 elementos: {element[:3]}")
    elif isinstance(element, dict):
        keys = list(element.keys())
        print(f"  - Primeras 5 claves: {keys[:5]}")
    else:
        # Para strings u otros tipos simples, mostrar una vista previa
        preview = str(element)[:200]
        print(f"  - Vista previa: {preview}...")
    
    print()

print("=== ANÁLISIS COMPLETADO ===")

=== ANÁLISIS DETALLADO DE RESULTADOS ===

Resultado de secsgml:
  - Tipo: <class 'tuple'>
  - Longitud: 2

Elemento 1:
  - Tipo: <class 'dict'>
  - Longitud: 26
  - Primeras 5 claves: [b'sec-document', b'sec-header', b'acceptance-datetime', b'accession-number', b'type']

Elemento 2:
  - Tipo: <class 'list'>
  - Longitud: 1
  - Primeros 3 elementos: [b'<?xml version=\'1.0\' encoding=\'ASCII\'?>\n<html xmlns="http://www.w3.org/1999/xhtml" xmlns:xs="http://www.w3.org/2001/XMLSchema-instance" xmlns:xsi="http://www.w3.org/2001/XMLSchema-instance" xmlns:xlink="http://www.w3.org/1999/xlink" xmlns:xbrli="http://www.xbrl.org/2003/instance" xmlns:xbrldi="http://xbrl.org/2006/xbrldi" xmlns:xbrldt="http://xbrl.org/2005/xbrldt" xmlns:iso4217="http://www.xbrl.org/2003/iso4217" xmlns:ix="http://www.xbrl.org/2013/inlineXBRL" xmlns:ixt="http://www.xbrl.org/inlineXBRL/transformation/2015-02-26" xmlns:ixt-sec="http://www.sec.gov/inlineXBRL/transformation/2015-08-31" xmlns:link="http://www.xbrl.org/2003/l